# Energy market analysis

## Instantiate the client

In [1]:
from esios import ESIOSClient
client = ESIOSClient()
endpoint = client.endpoint(name='indicators')

## Compose table

### List indicators

In [2]:
indicators = [600, 612, 613, 614, 615, 616, 617, 618]

### Iterate to get all indicators

In [4]:
dfs = []

for indicator in indicators:
    indicator = endpoint.select(indicator)
    df = indicator.historical(start='2024-01-01', end='2024-01-07', geo_ids=[3]).reset_index()
    dfs.append(df)

### Combine all data

In [5]:
from functools import reduce
import pandas as pd

df_base = reduce(lambda x, y: pd.merge(x, y, on=['datetime','geo_id', 'geo_name'], how='outer'), dfs)
df_base

,datetime,600,geo_id,geo_name,612,613,614,615,616,617,618
0,2024-01-01 00:00:00+01:00,63.33,3,España,54.33,57.00,63.32,NaN,NaN,NaN,NaN
1,2024-01-01 01:00:00+01:00,50.09,3,España,45.54,46.54,50.09,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
166,2024-01-07 22:00:00+01:00,91.17,3,España,88.57,91.17,91.17,95.17,93.00,100.02,87.61
167,2024-01-07 23:00:00+01:00,83.86,3,España,83.86,83.86,85.00,89.41,87.86,90.47,81.71


### Filter by geo

In [6]:
df = df_base.query('geo_name == "España"').drop(columns=['geo_id', 'geo_name'])
df

,datetime,600,612,613,614,615,616,617,618
0,2024-01-01 00:00:00+01:00,63.33,54.33,57.00,63.32,NaN,NaN,NaN,NaN
1,2024-01-01 01:00:00+01:00,50.09,45.54,46.54,50.09,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
166,2024-01-07 22:00:00+01:00,91.17,88.57,91.17,91.17,95.17,93.00,100.02,87.61
167,2024-01-07 23:00:00+01:00,83.86,83.86,83.86,85.00,89.41,87.86,90.47,81.71


### Temporal column as index

In [7]:
df_base = df.set_index('datetime')
df_base

,600,612,613,614,615,616,617,618
datetime,,,,,,,,
2024-01-01 00:00:00+01:00,63.33,54.33,57.00,63.32,NaN,NaN,NaN,NaN
2024-01-01 01:00:00+01:00,50.09,45.54,46.54,50.09,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2024-01-07 22:00:00+01:00,91.17,88.57,91.17,91.17,95.17,93.00,100.02,87.61
2024-01-07 23:00:00+01:00,83.86,83.86,83.86,85.00,89.41,87.86,90.47,81.71


### Export to Excel

In [8]:
df_base.tz_convert(None).to_excel('markets.xlsx')